# Append History Data
In this notebook, data from the `pseudo-absence generation` step is further processed to add 95 days history day for all temporal variables.

To achieve this, the NASA dataset is written to a database, then queries are made to fetch data of interest.

### Create Database

In [1]:
import pandas as pd
import sqlalchemy as db
from dateutil.relativedelta import relativedelta 
import glob
import xarray
import multiprocessing as mp
from functools import partial
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
# define variables

#path to NASA data
NASA_basePath = '/mnt/disks/nasa/NASA' 

temporal_variables = [
    'AvgSurfT_inst', 
    'Albedo_inst', 
    'SoilMoi0_10cm_inst', 
    'SoilMoi10_40cm_inst', 
    'SoilTMP0_10cm_inst', 
    'SoilTMP10_40cm_inst', 
    'Tveg_tavg', 
    'Wind_f_inst', 
    'Rainf_f_tavg', 
    'Tair_f_inst',
    'Qair_f_inst', 
    'Psurf_f_inst' 
]

In [4]:
# geo location gridding

resx, resy = (0.25, 0.25)

lat_to_bucket_id = lambda x: int((x+90)/resy)
lon_to_bucket_id = lambda x: int((x+180)/resx)

bucket_id_to_lat = lambda x: (x*resy) - 90
bucket_id_to_lon = lambda x: (x*resx) - 180

# date arithmetic

def add_days(current_index, days):
    return (pd.to_datetime(current_index[0]) + relativedelta(days=days), current_index[1], current_index[2])

In [6]:
# setting up database and table

table_name = "nasa_noah_data"
engine = db.create_engine('sqlite:///NASA_GLDAS_NOAH025_3H.db')
connection = engine.connect()
metadata = db.MetaData()
nasa_noah_data = db.Table(table_name, metadata, autoload=True, autoload_with=engine)

In [7]:
# Do THIS ONLY ONCE
# writing all NASA data to database

db_start_date = pd.to_datetime("2000-01-01")
db_end_date = pd.to_datetime("2021-12-31")

current_date = db_start_date
j = 0
while current_date <= db_end_date:
    if current_date.is_year_start:
        print(current_date)
    year, month, day = list(map(int, str(current_date.date()).split('-')))
    base_name = f"{NASA_basePath}/GLDAS_NOAH025_3H.A{year}{str(month).zfill(2)}"
    files_pattern = f"{base_name}{str(day).zfill(2) }*.nc4"
    try:
        data = xarray.open_mfdataset(files_pattern, parallel=True)
        data = data.mean(dim="time", skipna=True)
        data = data[temporal_variables].to_dataframe().dropna(axis=0, how='all').reset_index()
        data['lat_bucket_id'] = data['lat'].apply(lat_to_bucket_id)
        data['lon_bucket_id'] = data['lon'].apply(lon_to_bucket_id)
        data['year'] = year
        data['month'] = month
        data['day']  = day
        data["date"] = pd.to_datetime(data[['month', 'day', 'year']])
        data.index += j
        data.to_sql(table_name, engine, if_exists='append')
        j = data.index[-1] + 1
    except:
        print(f"Cannot read {current_date} data")
    current_date += relativedelta(days=1)

### Append History 

In [9]:
csv_filepath = 'train_val_random_v0.csv'
data = pd.read_csv(csv_filepath)

In [10]:
# February has <= 28 days
data.loc[((data['month']==2) & (data['day'] > 28)), 'day'] = 28
data["date"] = pd.to_datetime(data[['month', 'day', 'year']])
data["observation_date"] = data["date"]
data['lat_bucket_id'] = data['y'].apply(lat_to_bucket_id)
data['lon_bucket_id'] = data['x'].apply(lon_to_bucket_id)

In [11]:
stats = data[['lat_bucket_id', 'lon_bucket_id']].describe()
lat_min, lon_min = stats.loc['min']
lat_max, lon_max = stats.loc['max']
stats

,lat_bucket_id,lon_bucket_id
count,16159.000000,16159.000000
mean,435.691627,680.946284
std,11.376503,20.807735
min,401.000000,652.000000
25%,430.000000,665.000000
50%,436.000000,673.000000
75%,442.000000,695.000000
max,468.000000,736.000000


In [12]:
data = data[['date', 'lat_bucket_id', 'lon_bucket_id', 'x', 'y', 'presence', 'year', 'month', 'day', 'clay_0.5cm_mean', 'clay_5.15cm_mean', 'sand_0.5cm_mean', 'sand_5.15cm_mean', 'silt_0.5cm_mean', 'silt_5.15cm_mean', 'observation_date']]
data = data.set_index(['date', 'lat_bucket_id', 'lon_bucket_id'])
data

,,,x,y,presence,year,month,day,clay_0.5cm_mean,clay_5.15cm_mean,sand_0.5cm_mean,sand_5.15cm_mean,silt_0.5cm_mean,silt_5.15cm_mean,observation_date
date,lat_bucket_id,lon_bucket_id,,,,,,,,,,,,,
2000-01-11,442,658,-15.455833,20.581944,1,2000,1,11,0.134760,0.140545,0.651147,0.647941,0.214100,0.211528,2000-01-11
2000-01-29,461,676,-10.933333,25.383333,1,2000,1,29,0.161538,0.153232,0.615415,0.621584,0.223054,0.225189,2000-01-29
2000-01-04,437,655,-16.045278,19.471111,1,2000,1,4,0.178887,0.182475,0.554956,0.554456,0.229309,0.226969,2000-01-04
2000-01-02,435,660,-14.760833,18.966389,1,2000,1,2,0.195740,0.199793,0.574979,0.565655,0.229144,0.234419,2000-01-02
2000-01-17,442,658,-15.440278,20.643056,1,2000,1,17,0.134416,0.140512,0.653609,0.650924,0.211982,0.208577,2000-01-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-10-08,418,687,-8.125000,14.625000,0,2014,10,8,0.154982,0.164436,0.665226,0.657830,0.179781,0.177740,2014-10-08
2014-10-05,438,684,-8.875000,19.625000,0,2014,10,5,0.230420,0.228502,0.605109,0.604785,0.164489,0.166697,2014-10-05
2014-10-04,455,685,-8.625000,23.875000,0,2014,10,4,0.285872,0.279326,0.496537,0.500304,0.217598,0.220379,2014-10-04


In [13]:
def add_history_parallel(year, data):
    # February has <= 28 days
    data.loc[((data['month']==2) & (data['day'] > 28)), 'day'] = 28
    data["date"] = pd.to_datetime(data[['month', 'day', 'year']])
    data["observation_date"] = data["date"]
    data['lat_bucket_id'] = data['y'].apply(lat_to_bucket_id)
    data['lon_bucket_id'] = data['x'].apply(lon_to_bucket_id)
    data = data[['date', 'lat_bucket_id', 'lon_bucket_id', 'x', 'y', 'presence', 'method', 'year', 'month', 'day', 'clay_0.5cm_mean', 'clay_5.15cm_mean', 'sand_0.5cm_mean', 'sand_5.15cm_mean', 'silt_0.5cm_mean', 'silt_5.15cm_mean', 'observation_date']]
    data = data.set_index(['date', 'lat_bucket_id', 'lon_bucket_id'])
    

    start_date = str((relativedelta(days=-95) + pd.to_datetime(f"{year}-01-01")).date())
    end_date = str((relativedelta(days=365) + pd.to_datetime(f"{year}-01-01")).date())
    print(f"Year -> From: {start_date}, To: {end_date}")
    query = db.select([nasa_noah_data]).where(db.and_(
        nasa_noah_data.columns.date >= start_date, 
        nasa_noah_data.columns.date <= end_date,
        nasa_noah_data.columns.lat_bucket_id >= 347,
        nasa_noah_data.columns.lat_bucket_id <= 504,
        nasa_noah_data.columns.lon_bucket_id >= 619,
        nasa_noah_data.columns.lon_bucket_id <= 924,
    ))
    query_result = pd.read_sql_query(query, engine).set_index(['date', 'lat_bucket_id', 'lon_bucket_id'])
    subset = data[data['year'] == year]
    for days in range(0, 96):
        indices = subset['observation_date'].index.map(lambda row: add_days(row, days=-days))
        subset_day_x = query_result.reindex(indices)
        for variable in temporal_variables:
            subset[f"{variable}_{days}"] = list(subset_day_x[variable])
    return subset

In [14]:
filepaths = [
#     'v4_2/test_gen_v4_2.csv', 
    'v4_2/train_val_gen_v4_2.csv', 
#     'v3/test_ep_random_v3.csv', 
#     'v3/train_val_ep_random_v3.csv', 
#     'train_val_ep_kmeans_v2.csv', 
#     'test_ep_kmeans_v2.csv'
]

for filepath in filepaths:
    data = pd.read_csv(filepath)
    unique_years = data['year'].unique()

    with mp.Pool(3) as p:
        results = p.map(partial(add_history_parallel, data=data), unique_years)

    output = pd.concat(results).reset_index(drop=True)
    output.to_csv(f"{os.path.splitext(filepath)[0]}_full.csv")

Year -> From: 1999-09-28, To: 2000-12-31
Year -> From: 2001-09-28, To: 2003-01-01
Year -> From: 2003-09-28, To: 2004-12-31
Year -> From: 2000-09-28, To: 2002-01-01
Year -> From: 2002-09-28, To: 2004-01-01
Year -> From: 2004-09-28, To: 2006-01-01
Year -> From: 2005-09-28, To: 2007-01-01
Year -> From: 2007-09-28, To: 2008-12-31
Year -> From: 2009-09-28, To: 2011-01-01
Year -> From: 2006-09-28, To: 2008-01-01
Year -> From: 2008-09-28, To: 2010-01-01
Year -> From: 2010-09-28, To: 2012-01-01
Year -> From: 2011-09-28, To: 2012-12-31
Year -> From: 2013-09-28, To: 2015-01-01
Year -> From: 2012-09-28, To: 2014-01-01
